### Using Selenium to Web Scrape from Forex Factory (working)

Few notes:
- Macroeconomic data is web scrapped from Forex Factory
- Events scrapped can be changed based on conditions (High impact, CPI, PMI etc.)
- Program crashes after running a while, but all data previously scrapped during running is saved into a dataframe


UPDATED AS OF  <u>June 1 2022 <u>

#### Import Statements

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium_stealth import stealth
import time

#### Get specific event and corresponding historical data based on condition, and event descriptions
Conditions:
- all China events
- all CPI related events
- all PMI related events
- high impact events

In [3]:
# Switch to Microsoft Edge driver
#driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))

# Chrome options to bypass 
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
#options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#include Adblocker to prevent ads, not required but sometimes helps to block the pop-up google ads which causes the program to crash
path_to_extension = r'C:\Users\raych\Desktop\4.46.2_0'
options.add_argument('load-extension=' + path_to_extension)


driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver.create_options()

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

# Choose which week/month from forexfactory to webscrape from
#url = 'https://www.forexfactory.com/calendar?month=may.2022'
#url = 'https://www.forexfactory.com/calendar?week=apr24.2022'
#url = 'https://www.forexfactory.com/calendar?week=may22.2022'
#url = 'https://www.forexfactory.com/calendar?week=may15.2022'
#url = 'https://www.forexfactory.com/calendar?week=may1.2022'
#url = 'https://www.forexfactory.com/calendar?day=may24.2022'
#url = 'https://www.forexfactory.com/calendar?week=may26.2022'
url = 'https://www.forexfactory.com/calendar?day=jun1.2022'


# Webscraping code starts here
try:
    # Start driver
    driver.get(url)
    time.sleep(2)

    # Get the calendar table in webpage
    table = driver.find_element(By.CLASS_NAME, "calendar__table")
    
    row_date = []
    powerbi_links = []
    total_row_data = []

    # Start of first for loop, iterate over each table row
    for row in table.find_elements(By.TAG_NAME, "tr"):
        
        # Assign row_date to the latest date
        if list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "date")])) != []:
            row_date = list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "date")]))
            row_date = row_date[0].split("\n", 1)[1] + ', 2022'
            
        # Include the specific conditions you want to impose to filter the events
        # Take note: Filter for CB Leading Index m/m is a temporary hardcode fix to prevent program from crashing
        if row.find_elements(By.CLASS_NAME, 'calendar__currency') == []:
            continue
        else:
            if ('CNY' not in row.text and 'CPI' not in row.text and 'PMI' not in row.text and row.find_elements_by_class_name(
                'calendar__impact--high') == [] ) or 'CB Leading Index m/m' in row.text:
                continue 
        #print(row.text)


        #obtain latest event details, and put them into a list
        row_data_currency = list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "calendar__currency")]))
        row_data_event = list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "calendar__event")]))
        row_data_actual = list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "calendar__actual")]))
        row_data_forecast = list(filter(None, [td.text for td in row.find_elements(By.CLASS_NAME, "calendar__forecast")]))
    
        row_data = [row_date] + row_data_currency + row_data_event + row_data_actual + row_data_forecast
        if row_data == [] or row_data_actual == []:
            continue
        
        
        print(row.get_attribute('data-eventid'))
        print(row_data)
        total_row_data.append(row_data)
        powerbi_links.append(url+"#detail="+ row.get_attribute('data-eventid'))

    # End of first loop
    # Print all the latest events filtered in the list
    print(total_row_data)


    # Second loop, to get historical data for each high impact event
    
    final_df = []
    counter = 0
    CURRENCY = 1
    TITLE = 2

    # Loop over power_bi links to extract historical data from each link
    for link in powerbi_links:
        Dates = []
        Actuals = []
        Forecasts = []
        
        # Open next link to be webscraped in new window
        driver.execute_script("window.open('');")
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        driver.get(link)

        # Get historical data for each high impact event by clicking 'more' button
        click_more = True
        while click_more:
            time.sleep(2)
            try:
                element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='flexBox calendarhistory pad']//*[contains(text(),'More')]")))
                driver.execute_script("arguments[0].click();", element)
                print('scraping...')
                    
            except TimeoutException:
                click_more = False
                print('timeout!')

        # Extracting the date, actuals, currency and event detail
        try:
            table = driver.find_element(By.XPATH, "//table[@class='alternating calendarhistory']")
            table_rows = table.find_element(By.XPATH, "//tr[@class='odd' or @class='even']")

            table_desc = driver.find_element(By.XPATH, "//table[@class='calendarspecs']")

            # Make dictionary of information table
            title_list = table_desc.find_elements(By.XPATH,"//td[@class='label calendarspecs__spec']")
            text_list = table_desc.find_elements(By.XPATH,"//td[@class='full calendarspecs__specdescription']")
            info_dict = {}
            for i in range(len(title_list)):
                info_dict[title_list[i].text] = text_list[i].text
            
            print(info_dict)


            info_desc = [info_dict.get('Measures')]
            notes_desc = [info_dict.get('FF Notes')]
            impt_desc = [info_dict.get('Why Traders\nCare')]


            #ignore, testing comments
            '''xpath = "//td[@class ='history calendarhistory__row calendarhistory__row--history' or 'actual calendarhistory__row calendarhistory__row--actual' or 'forecast calendarhistory__row calendarhistory__row--forecast']"
            #for td in table_rows.find_elements_by_xpath(xpath):
                print(td.text)
                Row.append(td.text) '''

            for td in table_rows.find_elements(By.XPATH, "//td[@class ='history calendarhistory__row calendarhistory__row--history']"):
                Dates.append(td.text)

            for td in table_rows.find_elements(By.XPATH, "//td[@class='actual calendarhistory__row calendarhistory__row--actual']"):
                Actuals.append(td.text)
            
            for td in table_rows.find_elements(By.XPATH, "//td[@class='forecast calendarhistory__row calendarhistory__row--forecast']"):
                Forecasts.append(td.text)
            
            #combine historical data into final_df
            final_df.append(total_row_data[counter] + info_desc + notes_desc + impt_desc)
            for i in range(len(Dates)):
                final_df.append([Dates[i], total_row_data[counter][CURRENCY], total_row_data[counter][TITLE], Actuals[i], Forecasts[i]] + 
                info_desc + notes_desc + impt_desc)

            counter += 1
        finally:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print(e)
finally:
    print('end')
    #print(powerbi_links)
    driver.quit()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\raych\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\raych\AppData\Local\Temp\ipykernel_8052\1467663690.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\raych\AppData\Local\Temp\ipykernel_8052\1467663690.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


122427
['Jun 1, 2022', 'EUR', 'Spanish Manufacturing PMI', '53.8', '51.9']
122429
['Jun 1, 2022', 'CHF', 'Manufacturing PMI', '60.0', '61.2']
122428
['Jun 1, 2022', 'EUR', 'Italian Manufacturing PMI', '51.9', '53.5']
122430
['Jun 1, 2022', 'EUR', 'French Final Manufacturing PMI', '54.6', '54.5']
122426
['Jun 1, 2022', 'EUR', 'German Final Manufacturing PMI', '54.8', '54.7']
122431
['Jun 1, 2022', 'EUR', 'Final Manufacturing PMI', '54.6', '54.4']
122432
['Jun 1, 2022', 'GBP', 'Final Manufacturing PMI', '54.6', '54.6']
[['Jun 1, 2022', 'EUR', 'Spanish Manufacturing PMI', '53.8', '51.9'], ['Jun 1, 2022', 'CHF', 'Manufacturing PMI', '60.0', '61.2'], ['Jun 1, 2022', 'EUR', 'Italian Manufacturing PMI', '51.9', '53.5'], ['Jun 1, 2022', 'EUR', 'French Final Manufacturing PMI', '54.6', '54.5'], ['Jun 1, 2022', 'EUR', 'German Final Manufacturing PMI', '54.8', '54.7'], ['Jun 1, 2022', 'EUR', 'Final Manufacturing PMI', '54.6', '54.4'], ['Jun 1, 2022', 'GBP', 'Final Manufacturing PMI', '54.6', '54.

### Data manipulation 

In [4]:
#final_df contains all the event information and historical data scrapped from above url link
final_df[-1]

['Apr 1, 2011',
 'GBP',
 'Final Manufacturing PMI',
 '57.1',
 '60.7',
 'Level of a diffusion index based on surveyed purchasing managers in the manufacturing industry;',
 "Above 50.0 indicates industry expansion, below indicates contraction. The 'Previous' listed is the 'Actual' from the Flash release and therefore the 'History' data will appear unconnected. There are 2 versions of this report released about a week apart – Flash and Final. The Flash release, which the source first reported in Nov 2019, is the earliest and thus tends to have the most impact;",
 "It's a leading indicator of economic health - businesses react quickly to market conditions, and their purchasing managers hold perhaps the most current and relevant insight into the company's view of the economy;"]

#### Transforms final_df into pandas dataframe, cleans data to be exported to csv

In [5]:
import pandas as pd
import numpy as np

len(final_df)
df = pd.DataFrame(final_df, columns = ['Date', 'Currency', 'Event', 'Actual', 'Forecast', 'Info', 'Notes', 'Importance'])
#print(df)

#account for data with no forecasts, and Notes with None values
df['Forecast'].replace(to_replace=[None], value = '0', inplace = True)
df['Notes'].replace(to_replace=[None], value = 'NA', inplace = True)
df['Forecast'].replace(to_replace=[''], value = '0', inplace = True)
df['Units'] = df['Forecast'].str[-1]


#drop NA values
df = df.dropna()

# Hardcoded to remove this specific event, not needed
df = df.drop(df[df['Event'] == 'MPC Official Bank Rate Votes'].index)

df['Actual'] = df['Actual'].str.strip('%')
df['Actual'] = df['Actual'].str.strip('<')
df['Actual'] = df['Actual'].map(lambda x: x.rstrip('K'))
df['Actual'] = df['Actual'].map(lambda x: x.rstrip('B'))
df['Actual'] = df['Actual'].map(lambda x: x.rstrip('M'))
df['Actual'] = df['Actual'].astype(float)

df['Forecast'] = df['Forecast'].str.strip('%')
df['Forecast'] = df['Forecast'].str.strip('<')
df['Forecast'] = df['Forecast'].map(lambda x: x.rstrip('K'))
df['Forecast'] = df['Forecast'].map(lambda x: x.rstrip('B'))
df['Forecast'] = df['Forecast'].map(lambda x: x.rstrip('M'))
df['Forecast'] = df['Forecast'].astype(float)


df['Units'] = np.where(
    (df['Units'] == '0') | (df['Units'] =='1') | (df['Units'] =='2') | (df['Units'] =='3') | (df['Units'] =='4') | (df['Units'] =='5') | (df['Units'] =='6') 
    | (df['Units'] =='7') | (df['Units'] == '8') | (df['Units'] =='9'), 'NA', df['Units'])

df.head()

,Date,Currency,Event,Actual,Forecast,Info,Notes,Importance,Units
0,"Jun 1, 2022",EUR,Spanish Manufacturing PMI,53.8,51.9,Level of a diffusion index based on surveyed p...,"Above 50.0 indicates industry expansion, below...",It's a leading indicator of economic health - ...,NA
1,"May 2, 2022",EUR,Spanish Manufacturing PMI,53.3,54.0,Level of a diffusion index based on surveyed p...,"Above 50.0 indicates industry expansion, below...",It's a leading indicator of economic health - ...,NA
2,"Apr 1, 2022",EUR,Spanish Manufacturing PMI,54.2,55.8,Level of a diffusion index based on surveyed p...,"Above 50.0 indicates industry expansion, below...",It's a leading indicator of economic health - ...,NA
3,"Mar 1, 2022",EUR,Spanish Manufacturing PMI,56.9,55.9,Level of a diffusion index based on surveyed p...,"Above 50.0 indicates industry expansion, below...",It's a leading indicator of economic health - ...,NA
4,"Feb 1, 2022",EUR,Spanish Manufacturing PMI,56.2,56.1,Level of a diffusion index based on surveyed p...,"Above 50.0 indicates industry expansion, below...",It's a leading indicator of economic health - ...,NA


df converted to csv, will be saved in directory

In [6]:
#Export csv to directory, change directory to where csv is to be placed
# Always change the file_name to a unique name according to the duration of scrapped days
new_directory = 'June1_data.csv'
df.to_csv(new_directory)

### Combining new data with master data


In [9]:
#merge current dataframe with new events
# WORKING FILE WILL BE OVERWRITTEN TAKE NOTICE
main_directory = 'Master_Data.csv'

df = pd.read_csv(new_directory)
df1 = pd.read_csv(main_directory)


df = df.drop(df.columns[0], axis = 1)
df1 = df1.drop(df1.columns[0], axis = 1)

#print(df1.size)
#print(df1.drop_duplicates().size)

df_merge = pd.concat([df1,df])
#print(df_merge.size)
df_merge = df_merge.drop_duplicates()
df_merge = df_merge.reset_index().drop(columns = 'index', axis = 1)
#print(df_merge.size)
df_merge

# Export to new csv
### ONLY UNCOMMENT WHEN WANT TO MERGE###
df_merge.to_csv(main_directory, mode = 'w+')
